<a href="https://colab.research.google.com/github/falloatti/HenryProyectoUno/blob/main/MODELO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#habilito lectura en Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from pandas_profiling import ProfileReport
from datetime import datetime, timedelta
from sklearn import metrics
import math
from sklearn.metrics import classification_report
from sklearn import linear_model
import pickle as pkl
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
movies = pd.read_parquet(r'/content/drive/MyDrive/ProyectosHenry/archivoEDA.parquet')

In [ ]:
movies.shape

(11013823, 58)

In [ ]:
# Obtiene las columnas como una lista
columnas = movies.columns.tolist()

# Imprime la lista de columnas
print(columnas)

['userId', 'score', 'id', 'title', 'year', 'duration_int', 'duration_type', 'scored', 'multiple_values', 'mark knight', 'cannis holder', 'jay chapman', 'moonbug entertainment', 'arthur van merwijk', 'maggie binkley', 'shah rukh khan', 'gene autry', 'akshay kumar', 'nicolas cage', 'united states', 'india', 'diftiempo', '13+', '16', '16+', '18+', '7+', 'ages_16_', 'ages_18_', 'all', 'all_ages', 'g', 'nc-17', 'not rated', 'not_rate', 'nr', 'pg', 'pg-13', 'r', 'tv-14', 'tv-g', 'tv-ma', 'tv-nr', 'tv-pg', 'tv-y', 'tv-y7', 'tv-y7-fv', 'unrated', 'ur', 'drama', 'comedy', 'documentaries', 'kids', 'horror, suspense', 'amazon', 'netflix', 'hulu', 'disney']


In [ ]:
pip install scikit-surprise


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366463 sha256=6a58e54c8a4511a4f4c071897c0811a823e2bbf41e3116a7f85948a41c545495
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

In [ ]:
moviesModelo1 = movies[['userId', 'id','score']]

In [ ]:

# Definir el rango de calificaciones
reader = Reader(rating_scale=(1, 5))

In [ ]:
data = Dataset.load_from_df(moviesModelo1[['userId', 'id','score']], reader)


In [ ]:
# Crear un conjunto de entrenamiento y prueba a partir del objeto Dataset
trainset, testset = train_test_split(data, test_size=0.25)

In [ ]:
# Crear un modelo de recomendación utilizando el algoritmo SVD
model = SVD()

In [ ]:
# Entrenar el modelo utilizando el conjunto de entrenamiento
model.fit(trainset)

In [ ]:
# Generar predicciones para el conjunto de prueba
predictions = model.test(testset)

In [ ]:
# Hacer una predicción para un  usuario y una  película 
userId= 782
id = 'ns123'
prediction = model.predict(userId, id)

# Evaluar la calificación predicha y devolver un mensaje correspondiente
if prediction.est >= 4:
    mensaje = "Tenes que verla!!", prediction.est
elif prediction.est >= 3 and prediction.est < 4:
    mensaje = "Te la recomendamos!", prediction.est
else:
    mensaje = "La neta wey? No la veas!", prediction.est

# Imprimir el mensaje correspondiente
print(mensaje)

('Te la recomendamos!', 3.652265727589211)


In [ ]:

# Guardar el modelo entrenado en un archivo utilizando pickle
with open('mi_modelo_entrenado.pkl', 'wb') as archivo:
    pkl.dump(model, archivo)

Obtencion de muestra potencial por tiempos de procesamientos para un modelo mas completo, con menos peliculas y usuarios, pero filtrando por importancia por ahora no lo vamos a usar

In [ ]:
# Filtrar los id que aparecen con mas de 500 calificaciones 
id_counts = movies['id'].value_counts()
popular_ids = id_counts[id_counts > 500].index
df_popular = movies[movies['id'].isin(popular_ids)]

# Agrupar por grupos de 10000 userId y contar los id únicos
users = df_popular.groupby('userId')['id'].nunique()

# ordenar los usuarios por el número de 'id' únicos que tienen
users = users.sort_values(ascending=False)

# seleccionar los 10000 usuarios con más 'id' únicos
top_users = users[:10000]

# obtener una muestra del dataframe que contenga solo los 'id' de los 10000 usuarios seleccionados
sample_df = df_popular[df_popular['userId'].isin(top_users.index)]

In [ ]:
unique_values = sample_df['id'].nunique()
print("La columna 'id' tiene", unique_values, "valores únicos.")
unique_values = sample_df['userId'].nunique()
print("La columna 'userId' tiene", unique_values, "valores únicos.")

La columna 'id' tiene 3788 valores únicos.
La columna 'userId' tiene 10000 valores únicos.


In [ ]:
sample_df.shape

(1020495, 58)

In [ ]:
sample_df.to_csv('smalldataset.csv')